In [ ]:
import pandas as pd
import mysql.connector
from google.cloud import bigquery
from google.oauth2 import service_account
#import pyarrow.csv as csv

In [ ]:
#Conectar banco de dados MySQL
mydb = mysql.connector.connect(
    host="34.151.255.31",
    user="root",
    password="1234",
    database="banco_case_gb"
)

In [ ]:
#Criar cursor
cursor = mydb.cursor()

In [ ]:
# Definir o comando de SELECT no SQL
sql_select = "SELECT * FROM banco_case_gb.raw_vendas"

In [ ]:
# executa comando SQL
cursor.execute(sql_select)
# Coletar os dados da consulta
result = cursor.fetchall()

In [ ]:
# Fechar a conexão com o banco de dados
mydb.close()

In [ ]:
# Criar um DataFrame a partir dos resultados da origem
df_origem = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [ ]:
df_origem

In [ ]:
# Definir o nome da tabela no BigQuery
table_name = "projeto-gb-382302.BancoDW.raw_vendas_dw"

In [ ]:
# Define as credenciais de acesso
key_path = "./GBQ.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

In [ ]:
# Cria o objeto client do BigQery
client = bigquery.Client(project="projeto-gb-382302", credentials=credentials)

In [ ]:
# Definir o schema da tabela do BigQuery
schema = [
    bigquery.SchemaField("ID_MARCA", "INTEGER"),
    bigquery.SchemaField("MARCA", "STRING"),
    bigquery.SchemaField("ID_LINHA", "INTEGER"),
    bigquery.SchemaField("LINHA", "STRING"),
    bigquery.SchemaField("DATA_VENDA", "DATE"),
    bigquery.SchemaField("QTD_VENDA", "INTEGER"),
]

In [ ]:
# Criar a tabela no BigQuery se não existir
table_ref = client.dataset("BancoDW").table("raw_vendas_dw")
try:
    table = client.get_table(table_ref)
except:
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)

In [ ]:
# Inserir os dados na tabela do BigQuery
job = client.load_table_from_dataframe(df_origem, table_ref)
job.result()

print("Dados inseridos com sucesso na tabela do BigQuery!")
